<a href="https://colab.research.google.com/github/ashioyajotham/Natural-Language-Processing/blob/main/Finetuning/Falcon/Fine_Tuning_Falcon_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git

!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1

In [3]:
# In our tutorial, we will use the Guanaco dataset, which constitutes a refined segment of the OpenAssistant dataset designed specifically for the training of versatile chatbots.
from datasets import load_dataset

# Specify the name of the dataset
dataset_name = "timdettmers/openassistant-guanaco"

# Load the dataset from the specified name and select the "train" split
dataset = load_dataset(dataset_name, split="train")

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
# We will be loading the Falcon 7B model, applying 4bit quantization to it, and then adding LoRA adapters to the model.
import torch

from transformers import FalconForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Defining the name of the Falcon model
model_name = "ybelkada/falcon-7b-sharded-bf16"

# Configuring the BitsAndBytes quantization
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.float16,
)

# Loading the Falcon model with quantization configuration
model = FalconForCausalLM.from_pretrained(
model_name,
quantization_config=bnb_config,
trust_remote_code=True
)

# Disabling cache usage in the model configuration
model.config.use_cache = False

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type RefinedWebModel to instantiate a model of type falcon. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
# Load the tokenizer for the Falcon 7B model with remote code trust
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the padding token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

In [8]:
# Import the necessary module for LoRA configuration
from peft import LoraConfig

# Define the parameters for LoRA configuration
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

# Create the LoRA configuration object
peft_config = LoraConfig(
lora_alpha=lora_alpha,
lora_dropout=lora_dropout,
r=lora_r,
bias="none",
task_type="CAUSAL_LM",
target_modules=[
"query_key_value",
"dense",
"dense_h_to_4h",
"dense_4h_to_h",
]
)

In [9]:
from transformers import TrainingArguments
# Define the directory to save training results
output_dir = "./results"

# Set the batch size per device during training
per_device_train_batch_size = 4

# Number of steps to accumulate gradients before updating the model
gradient_accumulation_steps = 4

# Choose the optimizer type (e.g., "paged_adamw_32bit")
optim = "paged_adamw_32bit"

# Interval to save model checkpoints (every 10 steps)
save_steps = 10

# Interval to log training metrics (every 10 steps)
logging_steps = 10

# Learning rate for optimization
learning_rate = 2e-4

# Maximum gradient norm for gradient clipping
max_grad_norm = 0.3

# Maximum number of training steps
max_steps = 50

# Warmup ratio for learning rate scheduling
warmup_ratio = 0.03

# Type of learning rate scheduler (e.g., "constant")
lr_scheduler_type = "constant"

# Create a TrainingArguments object to configure the training process
training_arguments = TrainingArguments(
output_dir=output_dir,
per_device_train_batch_size=per_device_train_batch_size,
gradient_accumulation_steps=gradient_accumulation_steps,
optim=optim,
save_steps=save_steps,
logging_steps=logging_steps,
learning_rate=learning_rate,
fp16=True,  # Use mixed precision training (16-bit)
max_grad_norm=max_grad_norm,
max_steps=max_steps,
warmup_ratio=warmup_ratio,
group_by_length=True,
lr_scheduler_type=lr_scheduler_type,
)

In [10]:
# Import the SFTTrainer from the TRL library
from trl import SFTTrainer

# Set the maximum sequence length
max_seq_length = 512

# Create a trainer instance using SFTTrainer
trainer = SFTTrainer(
model=model,
train_dataset=dataset,
peft_config=peft_config,
dataset_text_field="text",
max_seq_length=max_seq_length,
tokenizer=tokenizer,
args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

In [12]:
# Iterate through the named modules of the trainer's model
for name, module in trainer.model.named_modules():

# Check if the name contains "norm"
  if "norm" in name:
	  # Convert the module to use torch.float32 data type
	  module = module.to(torch.float32)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.


Step,Training Loss
10,1.321300
20,1.254100
30,1.342200
40,1.533200
50,1.772900


TrainOutput(global_step=50, training_loss=1.4447361755371093, metrics={'train_runtime': 1445.2735, 'train_samples_per_second': 0.554, 'train_steps_per_second': 0.035, 'total_flos': 1.0840608893184e+16, 'train_loss': 1.4447361755371093, 'epoch': 0.08})

In [25]:
!pip install text-generation

# [Falcon Streaming](https://github.com/andrewgcodes/FalconStreaming)

## Basic Streaming (Greedy)

In [28]:
input_text = "Tell a story about a magical fish in the UAE:"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 80  # Change this to your desired output length. Too long could cause an OOM Out of Memory error!
current_length = input_ids.shape[1]

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    output = model(next_input)
    next_token_logits = output.logits[:, -1, :]
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
    print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

    next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break



The fish was a magical fish, it could fly and swim at the same time. It was a very special fish, it could do anything. It could fly, it could swim, it could even walk on land. It was a very special fish, it was a magical fish.
The fish was a very special fish, it could

## Top K

In [27]:

import torch
import torch.nn.functional as F

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 100  # Change this to your desired output length
current_length = input_ids.shape[1]
k = 10  # Number of tokens to sample from. Adjust as necessary. Greater k = greater variability.

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    output = model(next_input)
    next_token_logits = output.logits[:, -1, :]

    # Apply top-k sampling
    indices_to_remove = next_token_logits < torch.topk(next_token_logits, k)[0][..., -1, None]
    next_token_logits[indices_to_remove] = float('-inf')
    next_token_id = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1)

    print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

    next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break

 In my story, the magical fish lived in a beautiful, turquoise sea in the Emirates, and was known for its unique ability to transform into gold and silver whenever it was taken out of the water. But one day, a greedy fisherman decided to catch the magical fish, and took it out of the sea. When he tried to take it home, however, his greedy nature got the best of him and he left it to die in a bucket

In [29]:
import torch

def beam_search(input_ids, model, tokenizer, beam_size, max_length):
    beam = [(input_ids, 0)]  # Each element in the beam is a tuple (input_ids, score)

    for step in range(max_length):
        new_beam = []
        print(f"\n=== Step {step + 1} ===")

        for beam_id, (prev_input_ids, prev_score) in enumerate(beam):
            output = model(prev_input_ids)
            next_token_logits = output.logits[:, -1, :]
            next_token_probs = torch.softmax(next_token_logits, dim=-1)

            top_probs, top_indices = torch.topk(next_token_probs, beam_size)

            for i in range(beam_size):
                next_input_ids = torch.cat([prev_input_ids, top_indices[0][i].unsqueeze(0).unsqueeze(0)], dim=-1)
                score = prev_score + torch.log(top_probs[0][i]).item()

                new_beam.append((next_input_ids, score))

        beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_size]

        # Print out the current state of each beam
        for beam_id, b in enumerate(beam):
            print(f"\nBeam {beam_id + 1} Score: {b[1]}")
            print(tokenizer.decode(b[0][0].cpu().tolist(), skip_special_tokens=True))

    return beam

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

beam_size = 5  # Adjust as necessary
max_length = 100  # Adjust as necessary

beam = beam_search(input_ids, model, tokenizer, beam_size, max_length)

# Print out the highest-scoring output
print("\n=== Final Output ===")
print(f"\nBest Beam Score: {beam[0][1]}")
print(tokenizer.decode(beam[0][0][0].cpu().tolist(), skip_special_tokens=True))


=== Step 1 ===

Beam 1 Score: -1.1929776668548584
Tell a story about a magical fish in the UAE:


Beam 2 Score: -2.9443275928497314
Tell a story about a magical fish in the UAE: The

Beam 3 Score: -2.9682817459106445
Tell a story about a magical fish in the UAE: 

Beam 4 Score: -3.7227840423583984
Tell a story about a magical fish in the UAE: the

Beam 5 Score: -3.9044997692108154
Tell a story about a magical fish in the UAE: A

=== Step 2 ===

Beam 1 Score: -3.5928494930267334
Tell a story about a magical fish in the UAE:
The

Beam 2 Score: -3.9619851112365723
Tell a story about a magical fish in the UAE:
A

Beam 3 Score: -3.9832725524902344
Tell a story about a magical fish in the UAE:
I

Beam 4 Score: -4.071118116378784
Tell a story about a magical fish in the UAE:
Once

Beam 5 Score: -4.19096565246582
Tell a story about a magical fish in the UAE:
“

=== Step 3 ===

Beam 1 Score: -4.293824404478073
Tell a story about a magical fish in the UAE:
Once upon

Beam 2 Score: -5.5030164718